In [1]:
# Imports
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import datetime
import time

# Override yfinance API
yf.pdr_override()

In [2]:
# Get tickers for all S&P 500 stocks and replace "." with "-" for compatibility with Yahoo Finance
sp500_tickers = si.tickers_sp500()
sp500_tickers = [ticker.replace(".", "-") for ticker in sp500_tickers]

In [8]:
# Define S&P 500 index
sp500_index = 'SPY'

# Define date range for stock data
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [9]:
# Create empty list to store relative returns for each stock
relative_returns = []

In [10]:
# Retrieve historical price data for the S&P 500 index
sp500_df = pdr.get_data_yahoo(sp500_index, start_date, end_date)
sp500_df['Percent Change'] = sp500_df['Adj Close'].pct_change()
sp500_returns = sp500_df['Percent Change'].cumprod()
sp500_return = sp500_returns.iloc[-1]

[*********************100%%**********************]  1 of 1 completed


In [11]:
sp500_returns

Date
2022-09-20             NaN
2022-09-21   -1.744383e-02
2022-09-22    1.465265e-04
2022-09-23   -2.455015e-06
2022-09-26    2.428682e-08
                  ...     
2023-09-13    0.000000e+00
2023-09-14    0.000000e+00
2023-09-15   -0.000000e+00
2023-09-18   -0.000000e+00
2023-09-19    0.000000e+00
Name: Percent Change, Length: 251, dtype: float64

In [12]:
# Iterate over all S&P 500 stocks to calculate their relative returns
for ticker in sp500_tickers:
    # Download historical data as CSV for each stock to speed up the process
    stock_df = pdr.get_data_yahoo(ticker, start_date, end_date)
    stock_df.to_csv(f'{ticker}.csv')
    
    # Calculate percent change column
    stock_df['Percent Change'] = stock_df['Adj Close'].pct_change()
    
    # Calculate the relative return with double weight for the most recent quarter
    stock_returns = stock_df['Percent Change'].cumprod()
    stock_return = (stock_returns.iloc[-10] * 2 + stock_returns.iloc[-63]) / 3 # Double weight for the most recent quarter
    relative_return = stock_return / sp500_return
    relative_returns.append(relative_return)
    
    # Print relative return for each stock
    print(f'Ticker: {ticker}; Relative Return against S&P 500: {relative_return}\n')
    
    # Pause for 1 second to avoid overloading the server with requests
    time.sleep(1)

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['-']: Exception('%ticker%: No timezone found, symbol may be delisted')


IndexError: single positional indexer is out-of-bounds

In [20]:
# Create dataframe with relative returns and corresponding RS ratings
rs_df = pd.DataFrame(list(zip(sp500_tickers, relative_returns)), columns=['Ticker', 'Relative Return'])

In [22]:
rs_df['RS_Rating'] = rs_df['Relative Return'].rank(pct=True) * 100

In [24]:
rs_df

,Ticker,Relative Return,RS_Rating
0,A,NaN,NaN
1,AAL,NaN,NaN
2,AAP,NaN,NaN
3,AAPL,NaN,NaN
4,ABBV,NaN,NaN
...,...,...,...
149,DXC,1.222658,76.056338
150,DXCM,0.370674,52.816901
151,EA,1.780410,86.619718
152,EBAY,0.547714,59.859155


In [23]:
# Print RS Ratings for all stocks
print(rs_df)

    Ticker  Relative Return  RS_Rating
0        A              NaN        NaN
1      AAL              NaN        NaN
2      AAP              NaN        NaN
3     AAPL              NaN        NaN
4     ABBV              NaN        NaN
..     ...              ...        ...
149    DXC         1.222658  76.056338
150   DXCM         0.370674  52.816901
151     EA         1.780410  86.619718
152   EBAY         0.547714  59.859155
153    ECL        -0.625660   7.042254

[154 rows x 3 columns]
